[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/notebook-share/blob/master/docs/pakistan_floods.ipynb)
[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/giswqs/notebook-share/blob/master/docs/pakistan_floods.ipynb)
[![Open in Planetary Computer](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/giswqs/notebook-share&urlpath=lab/tree/notebook-share/docs/pakistan_floods.ipynb&branch=master)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/giswqs/notebook-share/HEAD?labpath=docs%pakistan_floods.ipynb)

# Visualization and Analysis of Pakistan Floods 2022 Using Earth Engine and Geemap



## Introduction

From 14 June to October 2022, floods in Pakistan killed 1,739 people, and caused ₨ 3.2 trillion ($14.9 billion) of damage and ₨ 3.3 trillion ($15.2 billion) of economic losses. The flooding was the world's deadliest flood since the 2020 South Asian floods and described as the worst in the country's history. On 25 August, Pakistan declared a state of emergency because of the flooding.

**Sources:**
- Wikipedia: [2022 Pakistan floods](https://en.wikipedia.org/wiki/2022_Pakistan_floods)
- UNICEF: [Devastating floods in Pakistan](https://www.unicef.org/emergencies/devastating-floods-pakistan-2022)


## Installation

In [1]:
# !pip install geemap

## Import libraries

In [2]:
import ee
import geemap

# import geemap.foliumap as geemap

## Create an interactive map

In [3]:
Map = geemap.Map()
Map.split_map()
Map

## Search datastes

![](https://i.imgur.com/1ef0sDz.gif)

## Visualize datasets

In [4]:
Map = geemap.Map()

country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    ee.Filter.eq('country_na', 'Pakistan')
)

style = {'color': 'black', 'fillColor': '00000000'}
Map.addLayer(country.style(**style), {}, 'Pakistan')
Map.centerObject(country)

Map

## Create Landsat composites

In [5]:
Map = geemap.Map()

landsat_col_2021 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1')
    .filterDate('2021-08-01', '2021-09-30')
    .filterBounds(country)
)
landsat_2021 = ee.Algorithms.Landsat.simpleComposite(landsat_col_2021).clipToCollection(
    country
)

vis_params = {'bands': ['B6', 'B5', 'B4'], 'max': 128}
Map.addLayer(landsat_2021, vis_params, 'Landsat 2021')

In [6]:
landsat_col_2022 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1')
    .filterDate('2022-08-01', '2022-09-30')
    .filterBounds(country)
)
landsat_2022 = ee.Algorithms.Landsat.simpleComposite(landsat_col_2022).clipToCollection(
    country
)

Map.addLayer(landsat_2022, vis_params, 'Landsat 2022')
Map.centerObject(country)
Map

## Compare Landsat composites side by side

In [7]:
Map = geemap.Map()
Map.setCenter(68.4338, 26.4213, 7)

left_layer = geemap.ee_tile_layer(landsat_2021, vis_params, 'Landsat 2021')
right_layer = geemap.ee_tile_layer(landsat_2022, vis_params, 'Landsat 2022')

Map.split_map(
    left_layer, right_layer, left_label='Landsat 2021', right_label='Landsat 2022'
)
Map.addLayer(country.style(**style), {}, 'Pakistan')
Map

## Compute Normalized Difference Water Index (NDWI)

In [8]:
ndwi_2021 = landsat_2021.normalizedDifference(['B3', 'B5']).rename('NDWI')
ndwi_2022 = landsat_2022.normalizedDifference(['B3', 'B5']).rename('NDWI')

In [9]:
ndwi_vis = {'min': -1, 'max': 1, 'palette': 'ndwi'}

In [10]:
Map = geemap.Map()
Map.setCenter(68.4338, 26.4213, 7)

left_layer = geemap.ee_tile_layer(ndwi_2021, ndwi_vis, 'NDWI 2021')
right_layer = geemap.ee_tile_layer(ndwi_2022, ndwi_vis, 'NDWI 2022')

Map.split_map(left_layer, right_layer, left_label='NDWI 2021', right_label='NDWI 2022')
Map.addLayer(country.style(**style), {}, 'Pakistan')
Map

## Extract Landsat water extent

In [11]:
threshold = 0.1
water_2021 = ndwi_2021.gt(threshold)
water_2022 = ndwi_2022.gt(threshold)

In [12]:
Map = geemap.Map()
Map.setCenter(68.4338, 26.4213, 7)

Map.addLayer(landsat_2021, vis_params, 'Landsat 2021', False)
Map.addLayer(landsat_2022, vis_params, 'Landsat 2022', False)

left_layer = geemap.ee_tile_layer(
    water_2021.selfMask(), {'palette': 'blue'}, 'Water 2021'
)
right_layer = geemap.ee_tile_layer(
    water_2022.selfMask(), {'palette': 'red'}, 'Water 2022'
)

Map.split_map(
    left_layer, right_layer, left_label='Water 2021', right_label='Water 2022'
)
Map.addLayer(country.style(**style), {}, 'Pakistan')
Map

## Extract Landsat flood extent

In [13]:
flood_extent = water_2022.subtract(water_2021).gt(0).selfMask()

In [14]:
Map = geemap.Map()
Map.setCenter(68.4338, 26.4213, 7)

Map.addLayer(landsat_2021, vis_params, 'Landsat 2021', False)
Map.addLayer(landsat_2022, vis_params, 'Landsat 2022', False)

left_layer = geemap.ee_tile_layer(
    water_2021.selfMask(), {'palette': 'blue'}, 'Water 2021'
)
right_layer = geemap.ee_tile_layer(
    water_2022.selfMask(), {'palette': 'red'}, 'Water 2022'
)

Map.split_map(
    left_layer, right_layer, left_label='Water 2021', right_label='Water 2022'
)

Map.addLayer(flood_extent, {'palette': 'cyan'}, 'Flood Extent')
Map.addLayer(country.style(**style), {}, 'Pakistan')
Map

## Calculate Landsat flood area

In [15]:
area_2021 = geemap.zonal_stats(
    water_2021.selfMask(), country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(area_2021)

Computing statistics ...


,sum,wld_rgn,country_na,abbreviati,country_co
0,4205.678431,S Asia,Pakistan,Pak.,PK


In [16]:
area_2022 = geemap.zonal_stats(
    water_2022.selfMask(), country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(area_2022)

Computing statistics ...


,sum,wld_rgn,country_na,abbreviati,country_co
0,13145.027451,S Asia,Pakistan,Pak.,PK


In [17]:
flood_area = geemap.zonal_stats(
    flood_extent.selfMask(), country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(flood_area)

Computing statistics ...


,sum,wld_rgn,country_na,abbreviati,country_co
0,11065.72549,S Asia,Pakistan,Pak.,PK


## Create Sentinel-1 SAR composites

In [18]:
s1_col_2021 = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    .filterDate('2021-08-01', '2021-09-30')
    .filterBounds(country)
    .select('VV')
)

In [19]:
# s1_col_2021

In [20]:
s1_col_2022 = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    .filterDate('2022-08-01', '2022-09-30')
    .filterBounds(country)
    .select('VV')
)

In [21]:
# s1_col_2022

In [22]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
sar_2021 = s1_col_2021.reduce(ee.Reducer.percentile([20])).clipToCollection(country)
sar_2022 = s1_col_2022.reduce(ee.Reducer.percentile([20])).clipToCollection(country)
Map.addLayer(sar_2021, {'min': -25, 'max': -5}, 'SAR 2021')
Map.addLayer(sar_2022, {'min': -25, 'max': -5}, 'SAR 2022')
Map.centerObject(country)
Map

## Apply speckle filtering

In [23]:
col_2021 = s1_col_2021.map(lambda img: img.focal_median(100, 'circle', 'meters'))
col_2022 = s1_col_2022.map(lambda img: img.focal_median(100, 'circle', 'meters'))

In [24]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
sar_2021 = col_2021.reduce(ee.Reducer.percentile([20])).clipToCollection(country)
sar_2022 = col_2022.reduce(ee.Reducer.percentile([20])).clipToCollection(country)
Map.addLayer(sar_2021, {'min': -25, 'max': -5}, 'SAR 2021')
Map.addLayer(sar_2022, {'min': -25, 'max': -5}, 'SAR 2022')
Map.centerObject(country)
Map

## Compare Sentinel-1 SAR composites side by side

In [25]:
Map = geemap.Map()
Map.setCenter(68.4338, 26.4213, 7)

left_layer = geemap.ee_tile_layer(sar_2021, {'min': -25, 'max': -5}, 'SAR 2021')
right_layer = geemap.ee_tile_layer(sar_2022, {'min': -25, 'max': -5}, 'SAR 2022')

Map.split_map(
    left_layer, right_layer, left_label='Sentinel-1 2021', right_label='Sentinel-1 2022'
)
Map.addLayer(country.style(**style), {}, 'Pakistan')
Map

## Extract SAR water extent

In [26]:
threshold = -18
water_2021 = sar_2021.lt(threshold)
water_2022 = sar_2022.lt(threshold)

In [27]:
Map = geemap.Map()
Map.setCenter(68.4338, 26.4213, 7)

Map.addLayer(sar_2021, {'min': -25, 'max': -5}, 'SAR 2021')
Map.addLayer(sar_2022, {'min': -25, 'max': -5}, 'SAR 2022')

left_layer = geemap.ee_tile_layer(
    water_2021.selfMask(), {'palette': 'blue'}, 'Water 2021'
)
right_layer = geemap.ee_tile_layer(
    water_2022.selfMask(), {'palette': 'red'}, 'Water 2022'
)

Map.split_map(
    left_layer, right_layer, left_label='Water 2021', right_label='Water 2022'
)
Map.addLayer(country.style(**style), {}, 'Pakistan')
Map

## Extract SAR flood extent

In [28]:
flood_extent = water_2022.subtract(water_2021).gt(0).selfMask()

In [29]:
Map = geemap.Map()
Map.setCenter(68.4338, 26.4213, 7)

Map.addLayer(sar_2021, {'min': -25, 'max': -5}, 'SAR 2021')
Map.addLayer(sar_2022, {'min': -25, 'max': -5}, 'SAR 2022')

left_layer = geemap.ee_tile_layer(
    water_2021.selfMask(), {'palette': 'blue'}, 'Water 2021'
)
right_layer = geemap.ee_tile_layer(
    water_2022.selfMask(), {'palette': 'red'}, 'Water 2022'
)

Map.split_map(
    left_layer, right_layer, left_label='Water 2021', right_label='Water 2022'
)

Map.addLayer(flood_extent, {'palette': 'cyan'}, 'Flood Extent')
Map.addLayer(country.style(**style), {}, 'Pakistan')
Map

## Calculate SAR flood area

In [30]:
area_2021 = geemap.zonal_stats(
    water_2021.selfMask(), country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(area_2021)

Computing statistics ...


,sum,wld_rgn,country_na,abbreviati,country_co
0,68949.458824,S Asia,Pakistan,Pak.,PK


In [31]:
area_2022 = geemap.zonal_stats(
    water_2022.selfMask(), country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(area_2022)

Computing statistics ...


,sum,wld_rgn,country_na,abbreviati,country_co
0,59224.121569,S Asia,Pakistan,Pak.,PK


In [32]:
flood_area = geemap.zonal_stats(
    flood_extent.selfMask(), country, scale=1000, statistics_type='SUM', return_fc=True
)
geemap.ee_to_df(flood_area)

Computing statistics ...


,sum,wld_rgn,country_na,abbreviati,country_co
0,12263.835294,S Asia,Pakistan,Pak.,PK
